### cria tabelas para API

In [24]:
import mysql.connector
from mysql.connector import errorcode

class mysqlserver:
    # variaveis de entrada
    def __init__(self,user, password, host, database):
        self.__user = user
        self.__password = password
        self.__host = host
        self.__database = database

    def create_server_connection(self):
        conn = None
        try:
            conn = mysql.connector.connect(user=self.__user,password=self.__password,host=self.__host,database=self.__database)
            print("Conectado no banco MYSQL")
        except mysql.connector.Error as err:
            print(f"Error: '{err}'")
        return conn

    def create_table(self, conn):
        cur = conn.cursor()
        try:
            cur.execute(""" CREATE TABLE IF NOT EXISTS candidate(
                            candidate_id VARCHAR(255) PRIMARY KEY,
                            candidate_name VARCHAR(255),
                            party_affiliation VARCHAR(255),
                            biography TEXT,
                            campaign_platform TEXT,
                            photo_url TEXT) 
                        """)
            cur.execute(""" CREATE TABLE IF NOT EXISTS voters (
                            voter_id VARCHAR(255) primary key,
                            voter_name VARCHAR(255),
                            date_of_birth VARCHAR(255),
                            gender VARCHAR(255),
                            nationality VARCHAR(255),
                            registration_number VARCHAR(255),
                            address_street VARCHAR(255),
                            address_city VARCHAR(255),
                            address_state VARCHAR(255),
                            address_country VARCHAR(255),
                            address_postcode VARCHAR(255),
                            email VARCHAR(255),
                            phone_number VARCHAR(255),
                            cell_number VARCHAR(255),
                            picture TEXT,
                            registered_age int
            )""")
            cur.execute("""CREATE TABLE IF NOT EXISTS votes (
                        voter_id VARCHAR(255) UNIQUE,
                        candidate_id VARCHAR(255),
                        voting_time TIMESTAMP,
                        vote int DEFAULT 1,
                        primary key (voter_id, candidate_id)
                        ) 
                        """)
            conn.commit()
        except mysql.connector.Error as err:
                print(f"Error: '{err}'") 

 

In [25]:
x = mysqlserver('root','mysql','localhost','mydesenv')
conn = x.create_server_connection()
x.create_table(conn)

Conectado no banco MYSQL


### insert de dados da API

In [26]:
import mysql.connector
from mysql.connector import errorcode
import random
import requests

BASE_URL = 'https://randomuser.me/api/?nat=gb'
PARTIES = ["Management Party", "Savior Party", "Tech Republic Party"]
random.seed(42)


def generate_voter_data():
    response = requests.get(BASE_URL)
    if response.status_code == 200:
        user_data = response.json()['results'][0]
        return {
            "voter_id": user_data['login']['uuid'],
            "voter_name": f"{user_data['name']['first']} {user_data['name']['last']}",
            "date_of_birth": user_data['dob']['date'],
            "gender": user_data['gender'],
            "nationality": user_data['nat'],
            "registration_number": user_data['login']['username'],
            "address": {
                "street": f"{user_data['location']['street']['number']} {user_data['location']['street']['name']}",
                "city": user_data['location']['city'],
                "state": user_data['location']['state'],
                "country": user_data['location']['country'],
                "postcode": user_data['location']['postcode']
            },
            "email": user_data['email'],
            "phone_number": user_data['phone'],
            "cell_number": user_data['cell'],
            "picture": user_data['picture']['large'],
            "registered_age": user_data['registered']['age']
        }
    else:
        return "Error fetching data"

def generate_candidate_data(candidate_number, total_parties):
    response = requests.get(BASE_URL + '&gender=' + ('female' if candidate_number % 2 == 1 else 'male'))
    if response.status_code == 200:
        user_data = response.json()['results'][0]
        return {
            "candidate_id": user_data['login']['uuid'],
            "candidate_name": f"{user_data['name']['first']} {user_data['name']['last']}",
            "party_affiliation": PARTIES[candidate_number % total_parties],
            "biography": "A brief bio of the candidate.",
            "campaign_platform": "Key campaign promises or platform.",
            "photo_url": user_data['picture']['large']
        }
    else:
        return "Error fetching data"

class mysqlserver:
    # variaveis de entrada
    def __init__(self,user, password, host, database):
        self.__user = user
        self.__password = password
        self.__host = host
        self.__database = database

           
    def create_server_connection(self):
        conn = None
        try:
            conn = mysql.connector.connect(user=self.__user,password=self.__password,host=self.__host,database=self.__database)
            print("Conectado no banco MYSQL")
        except mysql.connector.Error as err:
            print(f"Error: '{err}'")
        return conn
    def insert_table(self,conn):
        cur = conn.cursor()
        for i in range(20):
            candidate = generate_candidate_data(i, total_parties=3)
            print(candidate)
            cur.execute("""
                        INSERT INTO candidate (candidate_id, candidate_name, party_affiliation, biography, campaign_platform, photo_url)
                        VALUES (%s, %s, %s, %s, %s, %s)
                        """, (
                candidate['candidate_id'], candidate['candidate_name'], candidate['party_affiliation'], candidate['biography'],
                candidate['campaign_platform'], candidate['photo_url']))
        for i in range(100):
            voter = generate_voter_data()
            print(voter)
            cur.execute("""
                                INSERT INTO voters (voter_id, voter_name, date_of_birth, gender, nationality, registration_number, address_street, address_city, address_state, address_country, address_postcode, email, phone_number, cell_number, picture, registered_age)
                                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s,%s,%s,%s,%s,%s,%s)
                                """,
                        (voter["voter_id"], voter['voter_name'], voter['date_of_birth'], voter['gender'],
                        voter['nationality'], voter['registration_number'], voter['address']['street'],
                        voter['address']['city'], voter['address']['state'], voter['address']['country'],
                        voter['address']['postcode'], voter['email'], voter['phone_number'],
                        voter['cell_number'], voter['picture'], voter['registered_age'])
                        )
            conn.commit()

In [27]:
x = mysqlserver('root','mysql','localhost','mydesenv')
conn = x.create_server_connection()
x.insert_table(conn)

Conectado no banco MYSQL
{'candidate_id': 'be78fe85-1158-4c99-8913-0a2001d337b6', 'candidate_name': 'Jacob Reyes', 'party_affiliation': 'Management Party', 'biography': 'A brief bio of the candidate.', 'campaign_platform': 'Key campaign promises or platform.', 'photo_url': 'https://randomuser.me/api/portraits/men/18.jpg'}
{'candidate_id': '39bcf957-7ade-4fba-ad0a-1dbd7557e341', 'candidate_name': 'Vicki Diaz', 'party_affiliation': 'Savior Party', 'biography': 'A brief bio of the candidate.', 'campaign_platform': 'Key campaign promises or platform.', 'photo_url': 'https://randomuser.me/api/portraits/women/83.jpg'}
{'candidate_id': 'b7bc1a81-62a6-478c-934d-ae781a30514d', 'candidate_name': 'Clyde Pena', 'party_affiliation': 'Tech Republic Party', 'biography': 'A brief bio of the candidate.', 'campaign_platform': 'Key campaign promises or platform.', 'photo_url': 'https://randomuser.me/api/portraits/men/47.jpg'}
{'candidate_id': '7dfe63fa-6fa2-4ca2-848e-0b84cc77d29b', 'candidate_name': 'Ros

In [60]:
x = mysqlserver('root','mysql','localhost','mydesenv','total_parties')
for i in range(20):
    x.generate_candidate_data(i, total_parties=3)
    print(x)
        


TypeError: mysqlserver.generate_candidate_data() got an unexpected keyword argument 'total_parties'

In [12]:
def generate_candidate_data(candidate_number, total_parties):
    response = requests.get(BASE_URL + '&gender=' + ('female' if candidate_number % 2 == 1 else 'male'))
    if response.status_code == 200:
        user_data = response.json()['results'][0]
        return {
            "candidate_id": user_data['login']['uuid'],
            "candidate_name": f"{user_data['name']['first']} {user_data['name']['last']}",
            "party_affiliation": PARTIES[candidate_number % total_parties],
            "biography": "A brief bio of the candidate.",
            "campaign_platform": "Key campaign promises or platform.",
            "photo_url": user_data['picture']['large']
        }
    else:
        return "Error fetching data"
    
for i in range(20):
    candidate = generate_candidate_data(i, total_parties=3)
    print(candidate)

{'candidate_id': '82704a26-3873-4db0-92e6-ea182e80ee9c', 'candidate_name': 'Dwayne Lee', 'party_affiliation': 'Management Party', 'biography': 'A brief bio of the candidate.', 'campaign_platform': 'Key campaign promises or platform.', 'photo_url': 'https://randomuser.me/api/portraits/men/11.jpg'}
{'candidate_id': '963754ae-5295-4c7f-8c67-2b7ea5e7eab2', 'candidate_name': 'Karen Medina', 'party_affiliation': 'Savior Party', 'biography': 'A brief bio of the candidate.', 'campaign_platform': 'Key campaign promises or platform.', 'photo_url': 'https://randomuser.me/api/portraits/women/88.jpg'}
{'candidate_id': '40787f0f-3be2-4eff-9cdd-1eb704db2397', 'candidate_name': 'Herbert Gardner', 'party_affiliation': 'Tech Republic Party', 'biography': 'A brief bio of the candidate.', 'campaign_platform': 'Key campaign promises or platform.', 'photo_url': 'https://randomuser.me/api/portraits/men/71.jpg'}
{'candidate_id': 'b59134a5-1be3-4315-bf95-2b3bc24ef823', 'candidate_name': 'Julia Perkins', 'party